https://1drv.ms/u/s!As7Ri8JT5N9AqBYjIWE1aez2kSW-

# Machine Translation using RNN GRU
This use-case provides a demo of how RNN and GRU can be used for machine translation of languages using seq2seq.
In our use-case, we will use spa.txt as training set which has a translation of English words to Spanish words. The database can be downloaded from the URL https://www.dropbox.com/s/poqpk4528x4nkp3/spa.txt


*Workflow:*

1. Understanding the problem
2. Collecting the data
3. Data preprocessing 
4. Build seq2seq Model using Grated Recurrent Unit (GRU)
5. Training the dataset
6. Translate



## 1. Understanding the problem 
The problem statement is to translate the English text into Spanish. For this purpose, we will be using RNN-GRU model using sequence modeling. 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


##Download data and put it in pandas DataFrame

In [2]:
!wget https://www.dropbox.com/s/poqpk4528x4nkp3/spa.txt #Download the datafile from this site

--2021-08-22 14:21:29--  https://www.dropbox.com/s/poqpk4528x4nkp3/spa.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/poqpk4528x4nkp3/spa.txt [following]
--2021-08-22 14:21:29--  https://www.dropbox.com/s/raw/poqpk4528x4nkp3/spa.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbfd168621a13f11697ca6d0e03.dl.dropboxusercontent.com/cd/0/inline/BUt4oDGKeJX8xlGsfRJjE301MXyPTKbKpn4YHjd2CxPwQ2iUI0U_zJx7hobjDaZQdy0NvNSSd-rAoxfz3I7FhNlO8c7wo1evbxJdXtptuSrV5CIwAj4_N1vftgKzNfJqFFLfBrD2OZ7ZI5fbXAxn0l6s/file# [following]
--2021-08-22 14:21:29--  https://ucbfd168621a13f11697ca6d0e03.dl.dropboxusercontent.com/cd/0/inline/BUt4oDGKeJX8xlGsfRJjE301MXyPTKbKpn4YHjd2CxPwQ2iUI0U_zJx7hobjDaZQdy0NvNSSd-rAoxfz3I7FhNlO8c7wo1evbxJdXtptu

In [3]:
fi = open("/content/spa.txt",encoding='utf-8',mode='r')
data = fi.read()

In [4]:
df=[]
for line in data.split('\n'):
  df.append(line.split('\t'))

In [5]:
df[1000]

['Be careful.', 'Â¡SÃ© cuidadoso!']

In [6]:
lines=pd.DataFrame(df)
lines.head(20)
lines = lines.iloc[:,:2]
lines.columns=['english','spanish']

In [7]:
lines.dropna(inplace=True)
lines.shape

(123770, 2)

In [8]:
lines.head()

english   spanish
0  Go.     Ve.     
1  Go.     Vete.   
2  Go.     Vaya.   
3  Go.     VÃ¡yase.
4  Hi.     Hola.

In [9]:
lines.tail()

english                                                                                                                                                                                                                                             spanish
123765  There are four main causes of alcohol-related death. Injury from car accidents or violence is one. Diseases like cirrhosis of the liver                                                                         cancer                                                                                                                                                                                                                                           
123766  There are mothers and fathers who will lie awake after the children fall asleep and wonder how they'll make the mortgage                                                                                        or pay their doctor's bills                                                                                                                                                                                                                      
123767  A carbon footprint is the amount of carbon dioxide pollution that we produce as a result of our activities. Some people try to reduce their carbon footprint because they are concerned about climate change.  Una huella de carbono es la cantidad de contaminaciÃ³n de diÃ³xido de carbono que producimos como producto de nuestras actividades. Algunas personas intentan reducir su huella de carbono porque estÃ¡n preocupados acerca del cambio climÃ¡tico.
123768  Since there are usually multiple websites on any given topic                                                                                                                                                    I usually just click the back button when I arrive on any webpage that has pop-up advertising. I just go to the next page found by Google and hope for something less irritating.                                                                
123769  If you want to sound like a native speaker                                                                                                                                                                      you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo.

In [10]:
#lines=lines[~pd.isnull(lines['english'])]

In [11]:
lines.shape

(123770, 2)

In [12]:
lines.drop_duplicates(inplace=True)
lines=lines.sample(n=120000,random_state=42)
lines.shape

(120000, 2)

##Clean the text data

In [13]:
# Lowercase all characters
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['spanish']=lines['spanish'].apply(lambda x: x.lower())

# Remove quotes
lines['english']=lines['english'].apply(lambda x: re.sub("'", '', x))
lines['spanish']=lines['spanish'].apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['spanish']=lines['spanish'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
lines['spanish']=lines['spanish'].apply(lambda x: x.translate(remove_digits))


# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['spanish']=lines['spanish'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['spanish']=lines['spanish'].apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines['spanish'] = lines['spanish'].apply(lambda x : 'START_ '+ x + ' _END')

In [14]:
lines.tail()

english                                                 spanish
22728   this seems too easy                        START_ esto parece demasiado fã¡cil _END              
108301  im sorry                                   START_ i dont let in people i dont know _END          
78015   we couldnt do anything for them            START_ no pudimos hacer nada por ellos _END           
101956  the policeman took the knife from the boy  START_ el policã­a le quitã³ el cuchillo al niã±o _END
25663   its better as it is                        START_ es mejor asã­ _END

In [15]:
### Get English and Spanish Vocabulary
all_eng_words=set()
for eng in lines['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_spanish_words=set()
for spa in lines['spanish']:
    for word in spa.split():
        if word not in all_spanish_words:
            all_spanish_words.add(word)

lines['length_eng_sentence']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_spa_sentence']=lines['spanish'].apply(lambda x:len(x.split(" ")))
lines.head()

english                                                                 spanish  length_eng_sentence  length_spa_sentence
112706  tom entered his office and saw a box on his desk  START_ tom entrã³ a su oficina y vio una caja sobre el escritorio _END  11                   14                 
39299   the trees were in a row                           START_ los ã¡rboles estaban en lã­nea _END                              6                    7                  
115199  my uncle went to mexico in                        START_ never to come back _END                                          6                    6                  
39449   this book is very heavy                           START_ este libro es muy pesado _END                                    5                    7                  
11509   im sure of this                                   START_ estoy bastante seguro _END                                       4                    5

In [16]:
type(all_eng_words)

set

In [17]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_spa_sentence']<=20]
print("maximum length of Spanish Sentence ",max(lines['length_spa_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

max_length_src=max(lines['length_spa_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

input_words = sorted(list(all_eng_words)) # vocabulory
target_words = sorted(list(all_spanish_words)) 
num_encoder_tokens = len(all_eng_words) 
num_decoder_tokens = len(all_spanish_words)
print(num_encoder_tokens, num_decoder_tokens)
num_decoder_tokens += 1 #for zero padding

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)]) # create dictionary : Word & Value "hello" : 1
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items()) # 
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
lines = shuffle(lines)
lines.head(10)

maximum length of Spanish Sentence  20
maximum length of English Sentence  20
13141 28639


english                                                                    spanish  length_eng_sentence  length_spa_sentence
19486   what will you wear                                           START_ â¿quã© llevarã¡s puesto _END                                        4                    5                  
41196   exercise is good for you                                     START_ el ejercicio te sienta bien _END                                    5                    7                  
13851   i am the same age                                            START_ yo tengo la misma edad _END                                         5                    7                  
94868   it wont be as easy to do as you think                        START_ no serã¡ tan fã¡cil de hacer como crees _END                        10                   10                 
10156   youre an angel                                               START_ â¡eres un ã¡ngel _END                                               3                    5                  
108708  the salesgirl couldnt open the cash register                 START_ la vendedora no pudo abrir la caja registradora _END                7                    10                 
45134   you should go home early                                     START_ deberã­as volver pronto a casa _END                                 5                    7                  
19343   we were very happy                                           START_ ãramos muy felices _END                                            4                    5                  
119689  it was tom who first suggested that this might be the cause  START_ tom fue el primero que sugiriã³ que esta podrã­a ser la causa _END  12                   14                 
78139   what did you eat for lunch today                             START_ â¿quã© almorzaste hoy _END                                          7                    5

In [18]:
X, y = lines['english'], lines['spanish']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((95880,), (23970,))

##Generator to generate the data for training

In [19]:
np.arange(0,1000,128)

array([  0, 128, 256, 384, 512, 640, 768, 896])

In [23]:
X_train[10000].split()

['whats', 'at', 'stake']

In [24]:
input_token_index["whats"]

12815

In [31]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()): # Select each word in sequence & get its integer representation
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1: # ignoring end token
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [32]:

# A Python program to generate squares from 1
# to 100 using yield and therefore generator
 
# An infinite generator function that prints
# next square number. It starts with 1
def nextSquare():
    i = 1
 
    # An Infinite loop to generate squares
    while True:
        yield i*i                
        i += 1 # Next execution resumes
                # from this point    
 
# Driver code
for num in nextSquare():
    if num > 100:
        break   
    print(num)

1
4
9
16
25
36
49
64
81
100


##Create the model

help : https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/

In [33]:
#Model

latent_dim=300
# Encoder
encoder_inputs = Input(shape=(None, ))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb) # encoder_outputs - output at each LSTM neuron, output at last neuron, internal state
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
# Set up the decoder, using `encoder_states` as initial state.


decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [34]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [35]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    3942300     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 300)    8592000     input_4[0][0]                    
____________________________________________________________________________________________

In [36]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 10

##Train the model

In [37]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
749/749 [==============================] - 208s 262ms/step - loss: 1.7962 - val_loss: 1.5217
Epoch 2/10
749/749 [==============================] - 196s 261ms/step - loss: 1.3737 - val_loss: 1.2842
Epoch 3/10
749/749 [==============================] - 198s 264ms/step - loss: 1.1783 - val_loss: 1.1568
Epoch 4/10
749/749 [==============================] - 195s 260ms/step - loss: 1.0485 - val_loss: 1.0753
Epoch 5/10
749/749 [==============================] - 204s 273ms/step - loss: 0.9501 - val_loss: 1.0201
Epoch 6/10
749/749 [==============================] - 202s 269ms/step - loss: 0.8743 - val_loss: 0.9837
Epoch 7/10
749/749 [==============================] - 207s 276ms/step - loss: 0.8116 - val_loss: 0.9554
Epoch 8/10
749/749 [==============================] - 207s 276ms/step - loss: 0.7659 - val_loss: 0.9405
Epoch 9/10
749/749 [==============================] - 206s 276ms/step - loss: 0.7278 - val_loss: 0.9285
Epoch 10/10
749/749 [==============================] - 208s 278m

In [38]:
model.save_weights('nmt_weights.h5')

##Generate Translation

In [39]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :]) # last word's index
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [42]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [44]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Spanish Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Spanish Translation:', decoded_sentence[:-4])

Input English sentence: youd be surprised what you can learn in a week
Actual Spanish Translation:  te sorprenderã­a lo que puedes aprender en una semana 
Predicted Spanish Translation:  te deseo que te ha llegado a la clase una vez 
